In [20]:
from tensorflow.keras.models import load_model
import tensorflowjs as tfjs

try:
    model = load_model('model3.h5')    
    output_dir = './tfjs_model'
    tfjs.converters.save_keras_model(model, output_dir)
    
    print(f"Modelo guardado en {output_dir}")
except Exception as e:
    print(f"Error durante la conversion del modelo: {e}")


Modelo guardado en ./tfjs_model


In [26]:
import requests
import numpy as np

symbol = 'BTCUSDT'  
interval = '1d'     
limit = 7   

url = f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&limit={limit}'

response = requests.get(url)
data = response.json()

last_7_prices = [float(entry[2]) for entry in data]

last_7_prices = np.array(last_7_prices).reshape(-1, 1)

print("Ultimos precios de Binance:")
print(last_7_prices)


Ultimos precios de Binance:
[[91850.  ]
 [91779.66]
 [91449.99]
 [92594.  ]
 [93905.51]
 [94831.97]
 [94977.  ]]


In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

# Load or define your scaler if you used one during training
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the sequence
scaled_input = scaler.fit_transform(last_7_prices)

# Reshape input to (1, sequence_length, features) as expected by the model
scaled_input = scaled_input.reshape((1, limit, 1)) 

# Predict the next value
predicted_scaled = model.predict(scaled_input)

# Inverse transform to get the actual price prediction
predicted_price = scaler.inverse_transform(predicted_scaled)

print(f"Predicted Price: {predicted_price[0][0]}")


[[0.11341334]
 [0.09347011]
 [0.        ]
 [0.32435689]
 [0.69620443]
 [0.95888018]
 [1.        ]]
Predicted Price: 94541.8828125


In [ ]:
import numpy as np

predicciones = last_7_prices.copy()
predicciones = np.append(predicciones, predicted_price[0][0])

predicciones = np.array(predicciones.flatten())

diferencias = np.diff(predicciones)

min_diff = np.min(diferencias)
max_diff = np.max(diferencias)
scaled_diferencias = (diferencias - min_diff) / (max_diff - min_diff)

scaled_subida = np.where(diferencias > 0, scaled_diferencias, 0)  
scaled_bajada = np.where(diferencias <= 0, scaled_diferencias, 0) 

total_scaled = np.sum(scaled_diferencias)

probabilidad_subida = np.sum(scaled_subida) / total_scaled
probabilidad_bajada = np.sum(scaled_bajada) / total_scaled

print(f"Probabilidad de subida: {probabilidad_subida * 100:.2f}%")
print(f"Probabilidad de bajada: {probabilidad_bajada * 100:.2f}%")


Probabilidad de subida: 84.73%
Probabilidad de bajada: 15.27%
